<a href="https://colab.research.google.com/github/Abdullahaltek7/blood-supply-demand-prediction/blob/main/student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
dataset=pd.read_csv("student_depression_dataset.csv")
dataset.drop('Degree', axis=1, inplace=True)

In [ ]:
# Map the values
encoding_map = {'healthy': 0, 'moderate': 1, 'unhealthy': 2}
dataset['Dietary Habits'] = dataset['Dietary Habits'].map(encoding_map)

# Save result
dataset.to_csv("encoded_file.csv", index=False)

In [ ]:
yes_no_columns = ['Have you ever had suicidal thoughts ?', 'Family History of Mental Illness']

for col in yes_no_columns:
    dataset[col] = dataset[col].map({'Yes': 1, 'No': 0})

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
dataset['Sleep Duration'] = dataset['Sleep Duration'].str.replace("'", "").str.strip()
encoder = OrdinalEncoder(categories=[['Less than 5 hours', '5-6 hours', '7-8 hours','Others','More than 8 hours']])
dataset['Sleep Duration Encoded'] = encoder.fit_transform(dataset[['Sleep Duration']])

In [ ]:
X=dataset.iloc[:,1:-1].values
y=dataset.iloc[:,-1].values
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[0])],remainder='passthrough')
X=np.array(ct.fit_transform(X))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print("X_train dtype:", X_train.dtype)
print("y_train dtype:", y_train.dtype)

X_train dtype: object
y_train dtype: float64


In [ ]:
import numpy as np

def convert_range_to_float(value):
    try:
        if isinstance(value, str) and '-' in value:
            low, high = value.split('-')[:2]
            low = float(low.strip())
            high = float(high.split()[0].strip())
            return (low + high) / 2
        return float(value)
    except:
        return 0.0  # Or np.nan (requires imputation later)

# Convert X_train safely
X_train = np.array([
    [convert_range_to_float(x) for x in row]
    for row in X_train.reshape(len(X_train), -1)  # Ensure 2D
])
X_train = X_train.astype('float32')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
clas = RandomForestClassifier()
clas.fit(X_train, y_train)

RandomForestClassifier()

In [ ]:
# Use the same convert_range_to_float function from earlier
def convert_range_to_float(value):
    try:
        if isinstance(value, str):
            if '-' in value:  # Handle ranges like "5-6 hours"
                low, high = value.split('-')[:2]
                low = float(low.strip())
                high = float(high.split()[0].strip())
                return (low + high) / 2
            elif 'More than' in value:  # Handle "More than 8 hours"
                return float(value.split()[-1])  # Extract "8"
            else:
                return float(value)  # Direct conversion
        return float(value)
    except:
        return 0.0  # Default value (or np.nan if imputing later)

# Apply to X_test (same as X_train)
X_test = np.array([
    [convert_range_to_float(x) for x in row]
    for row in X_test.reshape(len(X_test), -1)
])
X_test = X_test.astype('float32')

In [ ]:
y_pred = clas.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.7781759541300842
Precision: 0.7700931347947666
Recall: 0.7781759541300842
F1 Score: 0.7703807886622405


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [ ]:
from sklearn.impute import SimpleImputer

# Replace NaN with mean (or strategy='median', 'most_frequent')
imputer = SimpleImputer(strategy='mean')

# Fit on X_train, then transform both X_train and X_test
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)  # Use the same imputer!

/usr/local/lib/python3.11/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: [9]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: [9]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


In [ ]:
from sklearn.svm import SVC
classifier=SVC(kernel='rbf',random_state=0)
classifier.fit(X_train,y_train)

SVC(random_state=0)

In [ ]:
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.7867765633399032
Precision: 0.7755934448942844
Recall: 0.7867765633399032
F1 Score: 0.7680158350542385


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.linear_model import LogisticRegression
classifier1=LogisticRegression(random_state=0)
classifier1.fit(X_train,y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [ ]:
y_pred = classifier1.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.7842680523203727
Precision: 0.77164017867149
Recall: 0.7842680523203727
F1 Score: 0.7636067743675846


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
